In [ ]:
import pickle
import mne
from pathlib import Path
import numpy as np
import pandas as pd

In [ ]:
eeg_electrodes_32 = ['FP1', 'FP2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FT9', 'FC5', 'FC1', 'FC2', 'FC6', 'FT10', 'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'Oz', 'O2']
eeg_electrodes_29 = ['F1', 'F2', 'Fz', 'CP3', 'C3', 'FC3', 'FCz', 'Cz', 'CPz', 'FC4', 'C4', 'CP4', 'FC5', 'C5', 'CP5', 'FC1', 'C1', 'CP1', 'FC2', 'C2', 'CP2', 'FC6', 'C6', 'CP6', 'P4', 'P3', 'P1', 'Pz', 'P2']
common_electrodes = set(eeg_electrodes_32).intersection(set(eeg_electrodes_29))

In [ ]:
data = {'left_real': [], 'right_real': []}  # signals
data_indexes = {'left_real': [], 'right_real': []}
data_stat = {'subject': [], 'left': [], 'right': []}# subject_trial_ids

29 electrodes parsing

In [24]:
data_29_path = 'E:/YandexDisk/EEG/raw/MI_NN/29_chanels/Real/'
data_codes = {'left_real': 1.0, 'right_real': 2.0}
subject_id = 0
for subject_path in Path(data_29_path).iterdir():
    data_stat['subject'].append(f"S{subject_id}")
    with open(f'{subject_path}/data_full.pkl', "rb") as f:
        data_full = pickle.load(f, encoding="bytes")
    with open(f'{subject_path}/states_full.pkl', "rb") as f:
        states_full = pickle.load(f, encoding="bytes")
    for movement in data_codes:
        movement_ids = np.argwhere(states_full == data_codes[movement])
        diff_values = np.diff(movement_ids[:, 1])
        split_indexes_ids = np.where(diff_values > 1.0)[0]
        split_indexes = movement_ids[:, 1][split_indexes_ids]
        data_stat[movement[:-5]].append(len(split_indexes_ids) + 1)
        for trial_id in range(0, len(split_indexes_ids) + 1):
            if trial_id == 0:
                trial_start = movement_ids[0, 1]
                trial_end = movement_ids[split_indexes_ids[trial_id], 1] + 1
            elif trial_id == len(split_indexes_ids):
                trial_start = movement_ids[split_indexes_ids[trial_id - 1], 1]
                trial_end = movement_ids[-1, 1] + 1
            else:
                trial_start = movement_ids[split_indexes_ids[trial_id - 1] + 1, 1]
                trial_end = movement_ids[split_indexes_ids[trial_id], 1] + 1
            data[movement].append(data_full[:, trial_start:trial_end] * 10 ** 6)
            data_indexes[movement].append(f"S{subject_id}_T{trial_id}_{movement}_29")
    subject_id += 1

num_subjects_29 = subject_id - 1
mne_python_raw = mne.io.read_raw_fif(f'{subject_path}/mne_python_raw.fif')
sample_frequency_29 = mne_python_raw.info['sfreq']
electrodes_names_29 = mne_python_raw.info['ch_names']

Opening raw data file E:\YandexDisk\EEG\raw\MI_NN\29_chanels\Real\ZhA_0.1/mne_python_raw.fif...
    Range : 0 ... 150368 =      0.000 ...   150.368 secs
Ready.


Test filtration

In [22]:
import scipy
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

curr_data = data['left_real'][0]
denoised_data = butter_bandpass_filter(curr_data, 1.0, 40.0, sample_frequency_29)
np.savetxt('E:/YandexDisk/EEG/electrodes/cur.csv', curr_data, delimiter=",")
np.savetxt('E:/YandexDisk/EEG/electrodes/den.csv', denoised_data, delimiter=",")

32 electrodes parsing

In [7]:
from pymatreader import read_mat

data_32_path = 'E:/YandexDisk/EEG/raw/'
data_32_file = '1st_Day.mat'

curr_32_data = read_mat(data_32_path + data_32_file)

In [25]:
electrodes_names_32 = curr_32_data['subs_ica'][0]['right_real']['label']
sample_frequency_32 = curr_32_data['subs_ica'][0]['right_real']['fsample']
for subject_id in range(0, len(curr_32_data['subs_ica'])):
    data_stat['subject'].append(f"S{num_subjects_29 + 1 + subject_id}")
    for key in data_codes:
        data_stat[key[:-5]].append(len(curr_32_data['subs_ica'][subject_id][key]['trial']))
        for trial_id in range(0, len(curr_32_data['subs_ica'][subject_id][key]['trial'])):
            data[key].append(curr_32_data['subs_ica'][subject_id][key]['trial'][trial_id][:, 5500:9500] * 10 ** 6)
            data_indexes[key].append(f"S{num_subjects_29 + 1 + subject_id}_T{trial_id}_{key}_32")

Data only for common electrodes

In [26]:
electrodes_names = list(set(electrodes_names_32).intersection(set(electrodes_names_29)))
electrodes_32_ids = []
electrodes_29_ids = []
for common_electrode in common_electrodes:
    electrodes_32_ids.append(electrodes_names_32.index(common_electrode))
    electrodes_29_ids.append(electrodes_names_29.index(common_electrode))
data_common = {'left_real': [], 'right_real': []}
for movement in data_codes:
    for trial_id in range(0, len(data[movement])):
        if data_indexes[movement][trial_id].endswith('29'):
            data_common[movement].append(data[movement][trial_id][electrodes_29_ids, :])
        if data_indexes[movement][trial_id].endswith('32'):
            data_common[movement].append(data[movement][trial_id][electrodes_32_ids, :])

Table with full set of features

In [ ]:
from mne.time_frequency import psd_array_multitaper
from mne_features.univariate import compute_spect_entropy
from tqdm import tqdm
from sklearn.utils import shuffle

movements = list(data.keys())
freq_bands = [('Theta', 4, 8), ('Alpha', 8, 12), ('Beta', 12, 30), ('Gamma', 30, 45)]

features_dict = {}
features_dict_names_all = {'features': []}
features_dict_names_alpha = {'features': []}

if sample_frequency_29 == sample_frequency_32:
    sample_frequency = sample_frequency_32
else:
    print('Sample frequencies are different')

for movement in data:
    print(movement)

    for trial_id in tqdm(range(0, len(data_common[movement]))):
        curr_index = data_indexes[movement][trial_id]
        if 'index' not in features_dict:
            features_dict['index'] = [curr_index]
        else:
            features_dict['index'].append(curr_index)
        curr_data = data[movement][trial_id]

        for freq_band in freq_bands:
            band_name = freq_band[0]
            band_low = freq_band[1]
            band_high = freq_band[2]
            filtered_data = butter_bandpass_filter(curr_data, band_low, band_high, sample_frequency)

            spect_ent = compute_spect_entropy(sfreq=sample_frequency, data=filtered_data, psd_method='multitaper')

            for electrode_id in range(0, len(electrodes_names)):
                electrode_name = electrodes_names[electrode_id]

                psds, freqs = psd_array_multitaper(x=curr_data[electrode_id, :], sfreq=sample_frequency, fmin=band_low, fmax=band_high, adaptive=True, normalization='full', verbose=None)

                curr_psd = np.trapz(psds, freqs)
                curr_spect_ent = spect_ent[electrode_id]

                psd_electrode_key = f"{electrode_name}_{band_name}_PSD"
                se_electrode_key = f"{electrode_name}_{band_name}_SE"

                if trial_id == 0 and movement == movements[0]:
                    features_dict_names_all['features'].append(psd_electrode_key)
                    features_dict_names_all['features'].append(se_electrode_key)
                    if band_name == 'Alpha':
                        features_dict_names_alpha['features'].append(psd_electrode_key)
                if psd_electrode_key not in features_dict and se_electrode_key not in features_dict:
                    features_dict[psd_electrode_key] = [curr_psd]
                    features_dict[se_electrode_key] = [curr_spect_ent]
                else:
                    features_dict[psd_electrode_key].append(curr_psd)
                    features_dict[se_electrode_key].append(curr_spect_ent)

        for electrode_id in range(0, len(electrodes_names)):
            electrode_name = electrodes_names[electrode_id]

            psds, freqs = psd_array_multitaper(x=curr_data[electrode_id, :], sfreq=sample_frequency, fmin=7, fmax=13, adaptive=True, normalization='full', verbose=0)
            curr_paf = freqs[np.argmax(psds)]
            curr_iaf = np.average(freqs, weights=psds)

            paf_electrode_key = f"{electrode_name}_PAF"
            iaf_electrode_key = f"{electrode_name}_IAF"
            if trial_id == 0 and movement == movements[0]:
                features_dict_names_all['features'].append(paf_electrode_key)
                features_dict_names_all['features'].append(iaf_electrode_key)
            if paf_electrode_key not in features_dict and iaf_electrode_key not in features_dict:
                features_dict[paf_electrode_key] = [curr_paf]
                features_dict[iaf_electrode_key] = [curr_iaf]
            else:
                features_dict[paf_electrode_key].append(curr_paf)
                features_dict[iaf_electrode_key].append(curr_iaf)

        if 'class' not in features_dict:
            features_dict['class'] = [movement]
        else:
            features_dict['class'].append(movement)
        if 'subject' not in features_dict:
            features_dict['subject'] = [curr_index.split('_')[0]]
        else:
            features_dict['subject'].append(curr_index.split('_')[0])

features_names_all_df = pd.DataFrame.from_dict(features_dict_names_all)
features_names_all_df.to_excel('E:/YandexDisk/EEG/Dataframes/29_32/features.xlsx', header=True, index=False)

features_names_alpha_df = pd.DataFrame.from_dict(features_dict_names_alpha)
features_names_alpha_df.to_excel(f"{dataframe_path}/features_alpha.xlsx", header=True, index=False)

features_df = pd.DataFrame.from_dict(features_dict)
features_df = shuffle(features_df)
features_df.reset_index(inplace=True, drop=True)
features_df.to_excel('E:/YandexDisk/EEG/Dataframes/29_32/data.xlsx', header=True, index=False)

left_real


  0%|          | 0/413 [00:00<?, ?it/s][Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend S

In [18]:
stat_df = pd.DataFrame.from_dict(data_stat)
stat_df['total'] = stat_df['left'] + stat_df['right']
stat_df.loc['row_total', :] = stat_df.sum(numeric_only=True, axis=0)
stat_df.to_excel('E:/YandexDisk/EEG/Dataframes/29_32/stat.xlsx', header=True, index=False)